In [20]:
import pandas as pd
import os
import numpy as np
np.set_printoptions(precision=4)

import catboost
from catboost import *
from catboost import CatBoostClassifier, Pool, metrics, cv

import matplotlib.pyplot as plt
import geopandas as gpd

# other things: 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, roc_curve, auc, roc_auc_score, recall_score, f1_score, confusion_matrix, precision_score

!jupyter nbextension enable --py widgetsnbextension


Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [14]:
# in this notebook varya is trying to buld a "best-working" catboost classifier for the DF and FF catchments 
# then "deconstrust" that model for the shap values to see how different features affect the result of the model 
# PD? 


# 1. read data and lable cat features 
# 2. split to train and test 
# 3. find the right hyperparameters (grid search)
# 4. 



# https://medium.com/@gabrieltseng/interpreting-complex-models-with-shap-values-1c187db6ec83

### 1. read the data, defince X and y, defince categorical features for y 

In [21]:
# read the data: 

path = '/Users/varyabazilova/Desktop/alluvial_fans/dem_all/all_points_so_far/ml_model/data_current/'
df = pd.read_csv(path + '09082022dfclim_permafrost.csv', sep = ',', index_col = 0)

df = df[df.target.isin([1,0])]
df = df.drop(['DN', 'Name', 'mean_daylymonsoon_tp_x', 'region_name'], axis = 1)


In [35]:
y = df.target
X = df.drop(columns=['target'])


# # what values are categorical: 
# # region, glacier, avgtemp_belowzero, cont_permafrost, any_permafrost, cont_permafrost_frac50
X['glacier'] = X['glacier'].astype('category') # n 21
X['region'] = X['region'].astype('category') # n 12
X['avgtemp_belowzero'] = X['avgtemp_belowzero'].astype('category') # n 34
X['cont_permafrost'] = X['cont_permafrost'].astype('category') # n 51
X['any_permafrost'] = X['any_permafrost'].astype('category') # n 52
X['cont_permafrost_frac50'] = X['cont_permafrost_frac50'].astype('category') # n 54

# cat_features = list([12, 21, 34, 50, 51, 53])
cat_features = list(['glacier', 'region', 'avgtemp_belowzero', 'cont_permafrost', 'any_permafrost', 'cont_permafrost_frac50'])

In [43]:
# X.dtypes
# Just change the categorical variables to the string by .astype(str)
# and when you check the data types by using .dtypes you will see them as "objects"
# then pass the cat_features as a list of indices. That would work fine.

# Don't change the data type to the category by .astype('category')


X['glacier'] = X['glacier'].astype(str) # n 21
X['region'] = X['region'].astype(str) # n 12
X['avgtemp_belowzero'] = X['avgtemp_belowzero'].astype(str) # n 34
X['cont_permafrost'] = X['cont_permafrost'].astype(str) # n 51
X['any_permafrost'] = X['any_permafrost'].astype(str) # n 52
X['cont_permafrost_frac50'] = X['cont_permafrost_frac50'].astype(str) # n 54

cat_features = list([12, 21, 34, 50, 51, 53])
# cat_features = list(['glacier', 'region', 'avgtemp_belowzero', 'cont_permafrost', 'any_permafrost', 'cont_permafrost_frac50'])

### 2. classifier data prep: split to train and test, convert to pool data class 

In [44]:
# train test 

seed = 42
test_size = 0.3

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)


In [56]:
# X_train.dtypes

In [68]:
# pool

train_pool = Pool(data=X_train,
                  label=y_train,
                  cat_features=cat_features,
                 )

test_pool = Pool(data=X_test,
                 label=y_test,
                 cat_features=cat_features,
                 )

### 3. catboost classifier - build the model and identify the hyperparameters that are needed 

In [95]:
%%time
# default parameters:

params = {
    'iterations': 500,
    'learning_rate': 0.03,
    'eval_metric': metrics.Accuracy(),
    # 'eval_metric': metrics.AUC(),
    'random_seed': 42,
    'logging_level': 'Silent',
    'use_best_model': False
}

CPU times: user 47 µs, sys: 7 µs, total: 54 µs
Wall time: 60.8 µs


In [105]:

# ------ model ------
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=test_pool)

print('Simple model validation Accuracy: {:.4}'.format(
    accuracy_score(y_test, model.predict(X_test))
))
print('')



# ------ best model ------

best_model_params = params.copy()
best_model_params.update({
    'use_best_model': True
})

best_model = CatBoostClassifier(**best_model_params)
best_model.fit(train_pool, eval_set=test_pool);


print('Best model validation Accuracy: {:.4}'.format(
    accuracy_score(y_test, best_model.predict(X_test))
))

print('')
# print('all parameters for the "best model: \n', best_model.get_all_params())
print('parameters for the "best model: \n', best_model.get_params())
print(' ')
print('parameters for the "model: \n', model.get_params())


print(' ')
print('best model get best iteration: \n', best_model.get_best_iteration())


Simple model validation Accuracy: 0.9019

Best model validation Accuracy: 0.9112

parameters for the "best model: 
 {'iterations': 500, 'learning_rate': 0.03, 'random_seed': 42, 'use_best_model': True, 'logging_level': 'Silent', 'eval_metric': 'Accuracy'}
 
parameters for the "model: 
 {'iterations': 500, 'learning_rate': 0.03, 'random_seed': 42, 'use_best_model': False, 'logging_level': 'Silent', 'eval_metric': 'Accuracy'}
 
best model get best iteration: 
 478


In [133]:
# stop the training when the best model is reached: 

earlystop_params = params.copy()
earlystop_params.update({
    'use_best_model': True,
    'od_type': 'Iter',
    # 'od_wait': 
})

earlystop_model = CatBoostClassifier(**earlystop_params)
earlystop_model.fit(train_pool, eval_set=test_pool);


print('best_model validation Accuracy: {:.4}'.format(
    accuracy_score(y_test, best_model.predict(X_test))
))

print('earlystop model validation Accuracy: {:.4}'.format(
    accuracy_score(y_test, earlystop_model.predict(X_test))
))

print('\n parameters for the earlystop model: \n', earlystop_model.get_params())


print('\n earlystop model get best iteration:', earlystop_model.get_best_iteration())


best_model validation Accuracy: 0.9112
earlystop model validation Accuracy: 0.8785

 parameters for the earlystop model: 
 {'iterations': 500, 'learning_rate': 0.03, 'od_type': 'Iter', 'random_seed': 42, 'use_best_model': True, 'logging_level': 'Silent', 'eval_metric': 'Accuracy'}

 earlystop model get best iteration: 27


In [134]:
print('Simple model tree count: {}'.format(model.tree_count_))
print('Simple model validation accuracy: {:.4}'.format(
    accuracy_score(y_test, model.predict(X_test))
))
print('')

print('Early-stopped model tree count: {}'.format(earlystop_model.tree_count_))
print('Early-stopped model validation accuracy: {:.4}'.format(
    accuracy_score(y_test, earlystop_model.predict(X_test))
))

Simple model tree count: 500
Simple model validation accuracy: 0.9019

Early-stopped model tree count: 28
Early-stopped model validation accuracy: 0.8785


In [138]:
## hyperparameters tuning: 

hyperparams = {'depth'         :[2,4,6,8,10],
               'learning_rate' :[0.01,0.03,0.05,0.07,0.1],
               'l2_leaf_reg'  : [0, 2, 1],
               'iterations' : [28]
               }

In [147]:
# cv: None — Use the default three-fold cross-validation.
# 
Grid_CBC = GridSearchCV(estimator=best_model, param_grid = hyperparams, cv = 2, n_jobs=-1)

# Grid_CBC.fit(X_train, y_train)

Grid_CBC.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_test, y_test), 
#     logging_level='Verbose',  # you can uncomment this for text output
    plot=True
)


print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",Grid_CBC.best_estimator_)
print("\n The best score across ALL searched params:\n",Grid_CBC.best_score_)
print("\n The best parameters across ALL searched params:\n",Grid_CBC.best_params_)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

 Results from Grid Search 

 The best estimator across ALL searched params:

 The best score across ALL searched params:
 0.8875502008032129

 The best parameters across ALL searched params:
 {'depth': 2, 'iterations': 28, 'l2_leaf_reg': 2, 'learning_rate': 0.01}


In [145]:
# # 1. number of trees: (= iterations)
# # tune this first to 

# model = CatBoostClassifier(
#     # custom_loss=[metrics.Accuracy()],
#     random_seed=42,
#     logging_level='Silent',
#     use_best_model = True, 
#     loss_function='CrossEntropy', 
#     # custom_loss=['AUC', 'Accuracy']
# )

# model.fit(
#     train_data,
#     # cat_features=cat_features,
#     eval_set=test_data, 
# #     logging_level='Verbose',  # you can uncomment this for text output
#     plot=True
# )
 

In [66]:
parameters = {'depth'         :[2,4,6,8,10],
              'learning_rate' :[0.01,0.03,0.05,0.07,0.1],
              # 'iterations'    :[10,20,30,40,50,60,70,80,90,100]
              'iterations'   : [90, 100, 105, 110, 115], 
              'l2_leaf_reg'  :
             }

# cv: None — Use the default three-fold cross-validation.
# 
Grid_CBC = GridSearchCV(estimator=model, param_grid = parameters, cv = 2, n_jobs=-1)

# Grid_CBC.fit(X_train, y_train)

Grid_CBC.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_test, y_test), 
#     logging_level='Verbose',  # you can uncomment this for text output
    plot=True
)


print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",Grid_CBC.best_estimator_)
print("\n The best score across ALL searched params:\n",Grid_CBC.best_score_)
print("\n The best parameters across ALL searched params:\n",Grid_CBC.best_params_)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

 Results from Grid Search 

 The best estimator across ALL searched params:

 The best score across ALL searched params:
 0.8855421686746988

 The best parameters across ALL searched params:
 {'depth': 2, 'iterations': 95, 'learning_rate': 0.1}
MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))
MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))
MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))
MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))
MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))
MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))
MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))
MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))
MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))
MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))
MetricVisualizer(layout=Layout(align_self='stretch', he

Exception in thread Thread-36:
Traceback (most recent call last):
  File "/Users/varyabazilova/opt/anaconda3/envs/project1/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Users/varyabazilova/opt/anaconda3/envs/project1/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/varyabazilova/opt/anaconda3/envs/project1/lib/python3.7/site-packages/catboost/widget/ipythonwidget.py", line 49, in start
    self._update_data()
  File "/Users/varyabazilova/opt/anaconda3/envs/project1/lib/python3.7/site-packages/catboost/widget/ipythonwidget.py", line 70, in _update_data
    content = self._update_data_from_dir(path)
  File "/Users/varyabazilova/opt/anaconda3/envs/project1/lib/python3.7/site-packages/catboost/widget/ipythonwidget.py", line 100, in _update_data_from_dir
    with open(training_json, 'r') as json_data:
FileNotFoundError: [Errno 2] No such file or directory: 'catboost_info/catboost_training.json'



In [67]:
CatBoostClassifier?

Init signature:
CatBoostClassifier(
    iterations=None,
    learning_rate=None,
    depth=None,
    l2_leaf_reg=None,
    model_size_reg=None,
    rsm=None,
    loss_function=None,
    border_count=None,
    feature_border_type=None,
    per_float_feature_quantization=None,
    input_borders=None,
    output_borders=None,
    fold_permutation_block=None,
    od_pval=None,
    od_wait=None,
    od_type=None,
    nan_mode=None,
    counter_calc_method=None,
    leaf_estimation_iterations=None,
    leaf_estimation_method=None,
    thread_count=None,
    random_seed=None,
    use_best_model=None,
    best_model_min_trees=None,
    verbose=None,
    silent=None,
    logging_level=None,
    metric_period=None,
    ctr_leaf_count_limit=None,
    store_all_simple_ctr=None,
    max_ctr_complexity=None,
    has_time=None,
    allow_const_label=None,
    target_border=None,
    classes_count=None,
    class_weights=None,
    auto_class_weights=None,
    class_names=None,
    one_hot_max_size=Non

In [ ]:
# import CatBoostClassifier
# logloss - default(?)
# CrossEntropy - for probabilities 

model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.1,
    use_best_model= True,
    loss_function='CrossEntropy', 
    custom_loss=['AUC', 'Accuracy']
)


model.fit(
    train_data,                # pool thing - instead of X_train, y_train
    # cat_features=cat_features, # since the data is a pool - it already knows about it 
    eval_set=test_data,        # pool thing - instead of X_test, y_test
    verbose=False, 
    plot=True
)

print('Model is fitted: ' + str(model.is_fitted()))
print('Model params:')
print(model.get_params())